## Práctica


#### Inicialización del entorno y parámetros

In [ ]:
import simpy
import random
import statistics

NUM_RECEPCIONISTAS = 1
NUM_MEDICOS = 2
TIEMPO_REGISTRO_PROMEDIO = 2.0      
TIEMPO_CONSULTA_PROMEDIO = 7.0      
TASA_LLEGADA_PACIENTES = 5.0        
TIEMPO_SIMULACION = 120.0          

# Lista para almacenar los tiempos totales de permanencia
tiempos_de_espera_totales = []
